In [ ]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

train_data = np.loadtxt( "../input/digit-recognizer/train.csv", delimiter=",", skiprows=1)

x = (train_data[:, 1:])
y = (train_data[:, [0]])


In [ ]:
digits = 10
All_of_mnist_record = 42000
Epoch = 2000
imageID = 9
n_x = 784
n_h = 64
learning_rate = 4

X = x / 255
y = y.reshape(1, All_of_mnist_record)
Y_new = np.eye(digits)[y.astype('int32')]
Y_new = Y_new.T.reshape(digits, All_of_mnist_record)



X_train, X_test = X[:All_of_mnist_record].T, X[All_of_mnist_record:].T
Y_train, Y_test = Y_new[:,:All_of_mnist_record], Y_new[:,All_of_mnist_record:]

W1 = np.random.randn(n_h, n_x)
b1 = np.zeros((n_h, 1))
W2 = np.random.randn(digits, n_h)
b2 = np.zeros((digits, 1))

X = X_train
Y = Y_train


beta = .9
batch_size = 128
batches = -(-All_of_mnist_record // batch_size)



yPredicted = np.array(Y[:,imageID].T, dtype=float) 
print(yPredicted)
xPredicted = np.array(X[:,imageID].T, dtype=float) 
plt.imshow(X[:,imageID].reshape(28,28), cmap = matplotlib.cm.binary)
plt.axis("off")
plt.show()

def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

def loss(Y, Y_hat):
    L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
    L = -(1/All_of_mnist_record) * L_sum
    return L



def feed_forward(X, params):

    cache = {}

    cache["Z1"] = np.matmul(params["W1"], X) + params["b1"]
    cache["A1"] = sigmoid(cache["Z1"])
    cache["Z2"] = np.matmul(params["W2"], cache["A1"]) + params["b2"]
    cache["A2"] = np.exp(cache["Z2"]) / np.sum(np.exp(cache["Z2"]), axis=0)

    return cache

def back_propagate(X, Y, params, cache):

    dZ2 = cache["A2"] - Y
    dW2 = (1./m_batch) * np.matmul(dZ2, cache["A1"].T)
    db2 = (1./m_batch) * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.matmul(params["W2"].T, dZ2)
    dZ1 = dA1 * sigmoid(cache["Z1"]) * (1 - sigmoid(cache["Z1"]))
    dW1 = (1./m_batch) * np.matmul(dZ1, X.T)
    db1 = (1./m_batch) * np.sum(dZ1, axis=1, keepdims=True)

    back_propagate_res = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}

    return back_propagate_res


# initialization
params = { "W1": np.random.randn(n_h, n_x) * np.sqrt(1. / n_x),
           "b1": np.zeros((n_h, 1)) * np.sqrt(1. / n_x),
           "W2": np.random.randn(digits, n_h) * np.sqrt(1. / n_h),
           "b2": np.zeros((digits, 1)) * np.sqrt(1. / n_h) }

V_dW1 = np.zeros(params["W1"].shape)
V_db1 = np.zeros(params["b1"].shape)
V_dW2 = np.zeros(params["W2"].shape)
V_db2 = np.zeros(params["b2"].shape)


for i in range(Epoch):

    permutation = np.random.permutation(X_train.shape[1])
    X_train_shuffled = X_train[:, permutation]
    Y_train_shuffled = Y_train[:, permutation]

    for j in range(batches):

        begin = j * batch_size
        end = min(begin + batch_size, X_train.shape[1] - 1)
        X = X_train_shuffled[:, begin:end]
        Y = Y_train_shuffled[:, begin:end]
        m_batch = end - begin

        cache = feed_forward(X, params)
        back_propagate_res = back_propagate(X, Y, params, cache)

        V_dW1 = (beta * V_dW1 + (1. - beta) * back_propagate_res["dW1"])
        V_db1 = (beta * V_db1 + (1. - beta) * back_propagate_res["db1"])
        V_dW2 = (beta * V_dW2 + (1. - beta) * back_propagate_res["dW2"])
        V_db2 = (beta * V_db2 + (1. - beta) * back_propagate_res["db2"])

        params["W1"] = params["W1"] - learning_rate * V_dW1
        params["b1"] = params["b1"] - learning_rate * V_db1
        params["W2"] = params["W2"] - learning_rate * V_dW2
        params["b2"] = params["b2"] - learning_rate * V_db2

    
    if (i % 10 == 0):
      cache = feed_forward(X_train, params)
      train_cost = loss(Y_train, cache["A2"])
      print("Epoch {}: training cost = {}".format(i ,train_cost))
      #Z1 = np.matmul(params["W1"], xPredicted) + V_db1
      #A1 = sigmoid(Z1)
      #Z2 = np.matmul(params["W2"], A1) + V_db2
      #A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0)
      #predictions = np.argmax(A2, axis=0)
      #print(predictions)
      cache = feed_forward(X_test, params)
      predictions = np.argmax(cache["A2"], axis=0)
      #print(predictions)
      labels = np.argmax(Y_test, axis=0)
      #print(labels)

      #cache = feed_forward(xPredicted, params)
      #predictions = np.argmax(cache["A2"], axis=0)
      #print(predictions)
      #print((np.mean(predictions)))



print("Done.")